<a href="https://colab.research.google.com/github/Mennakurdi/Data-Engineering-Project/blob/colab-Menna/post__integration_%2Bfinal_analysis_%2B_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dashboard & Spatial Analysis 5. Which areas (zip codes) show consistently high crash density over time, and how do they relate to pedestrian injury counts? 6. At what times of day do most severe crashes (with at least one person killed or seriously injured) occur?

In [ ]:
# ----- FIX 3: DROP IRRELEVANT COLUMNS -----
columns_to_drop = [
    'location','on_street_name','off_street_name','cross_street_name',
    'contributing_factor_vehicle_2','contributing_factor_vehicle_3',
    'contributing_factor_vehicle_4','contributing_factor_vehicle_5',
    'vehicle_type_code2','vehicle_type_code_3','vehicle_type_code_4','vehicle_type_code_5',
    'unique_id','person_id','person_type','person_injury','vehicle_id',
    'person_age','ejection','emotional_status','bodily_injury',
    'position_in_vehicle','safety_equipment','ped_location','ped_action',
    'complaint','ped_role','contributing_factor_1','contributing_factor_2','person_sex'
]

merged_df = merged_df.drop(columns=columns_to_drop, errors='ignore')

# ----- FIX 4: RENAME CONFUSING COLUMNS -----
merged_df = merged_df.rename(columns={
    'crash_date_crash': 'crash_date',
    'crash_time_crash': 'crash_time'
})

# ----- FIX 5: CHECK MISSINGNESS -----
print("Missing crash_date:", merged_df['crash_date'].isna().mean())
print("Missing crash_time:", merged_df['crash_time'].isna().mean())


## 3. Post-Integration Cleaning
After merging, remove redundant columns, fix types, and clean new nulls.


In [ ]:
# 1️⃣ inspect missing values
merged_df.isna().mean().sort_values(ascending=False).head(15)

# 2️⃣ drop duplicated columns (_person that duplicate _crash)
dup_cols = [c for c in merged_df.columns
            if c.endswith('_person') and c.replace('_person','_crash') in merged_df.columns]
merged_df.drop(columns=dup_cols, inplace=True, errors='ignore')

# 3️⃣ numeric columns → proper types
num_cols = ['number_of_persons_injured','number_of_persons_killed']
for c in num_cols:
    if c in merged_df.columns:
        merged_df[c] = pd.to_numeric(merged_df[c], errors='coerce').fillna(0).astype(int)

# 4️⃣ remove negatives
for c in num_cols:
    merged_df.loc[merged_df[c]<0, c] = 0

# 5️⃣ confirm
merged_df.info()


### 3.x Post-Integration Missing Values & Consistency Check

After merging `crash` and `persons` on `collision_id`, we perform a quick audit of:

- **New missing values introduced by the join**  
- Potentially **redundant or duplicated columns**  
- Remaining **type mismatches** that might break the dashboard later

The following cell computes missing-value percentages and highlights the columns that still need attention.


In [ ]:
# Post-integration missing values analysis
na_ratio = merged_df.isna().mean().sort_values(ascending=False)
print("Top 20 columns by missing-value ratio after integration:")
display(na_ratio.head(20))

# Identify obviously redundant columns (same logical field with _x / _y suffixes)
dup_like = [c for c in merged_df.columns if c.endswith('_x') or c.endswith('_y')]
print("\nPotential duplicated columns from merge:", dup_like)


## 4. Export Clean Dataset
Keep only columns needed for the interactive app and export to CSV.


In [ ]:
print(merged_df.columns)


In [ ]:
selected_cols = [
    'collision_id','crash_date','crash_time','borough','zip_code',
    'latitude','longitude',
    'number_of_persons_injured','number_of_persons_killed',
    'number_of_pedestrians_injured','number_of_pedestrians_killed',
    'number_of_cyclist_injured','number_of_cyclist_killed',
    'number_of_motorist_injured','number_of_motorist_killed',
    'contributing_factor_vehicle_1','vehicle_type_code1'
]

# Keep only columns that actually exist in merged_df (for robustness)
selected_cols = [c for c in selected_cols if c in merged_df.columns]

df_site = merged_df[selected_cols].drop_duplicates(subset=['collision_id']).copy()

# Feature engineering for easier dashboard filters
df_site['crash_date'] = pd.to_datetime(df_site['crash_date'], errors='coerce')
df_site['crash_year'] = df_site['crash_date'].dt.year
df_site['crash_month'] = df_site['crash_date'].dt.to_period('M').astype(str)

# Extract hour if we have a time column
if 'crash_time' in df_site.columns:
    df_site['crash_hour'] = pd.to_datetime(df_site['crash_time'], errors='coerce').dt.hour

df_site.to_csv('/content/df_site.csv', index=False)
print("✅ Clean dataset exported as df_site.csv")
print("Final df_site columns:", df_site.columns.tolist())


### 4.x Final Dataset for Dashboard (`df_site`)

The exported `df_site` dataset is designed **specifically for the interactive website**:

- Includes **location** fields: `borough`, `zip_code`, `latitude`, `longitude`  
- Includes **severity measures**: `number_of_persons_injured`, `number_of_persons_killed`  
- Includes **injury-type fields** (if present): pedestrians, cyclists, motorists  
- Includes **cause & vehicle**: `contributing_factor_vehicle_1`, `vehicle_type_code1`  
- Includes **time features**: `crash_year`, `crash_month`, and (when available) `crash_hour`

These fields are enough to implement the required filters in the web app:

- Filter by **Borough** (borough)  
- Filter by **Year** (crash_year)  
- Filter by **Vehicle Type** (vehicle_type_code1)  
- Filter by **Contributing Factor** (contributing_factor_vehicle_1)  
- Filter by **Injury Type** (using counts per pedestrian/cyclist/motorist or total injuries)


In [ ]:
## 5. Verification
Quick sanity check on exported dataset.


In [ ]:
print("Rows & Cols →", df_site.shape)
display(df_site.head())
df_site.isna().mean().sort_values(ascending=False).head(10)


## Summary
- Both datasets cleaned, standardized, and merged on `collision_id`.
- Final dataset exported as df_site.csv for dashboard visualization.
- Total rows: 1000, columns: 11.


Led the integration of crashes and persons datasets via collision_id.
Designed and documented pre- and post-integration cleaning steps.
Prepared the final cleaned dataset (df_site) for the web application.